### rackcli - h4x^2 VCV Rack via command line

This is a catchall for any h4x that don't yet fit somewhere else, a lab's lab so to speak.

In [1]:
%run rackcli_core.ipynb


rackcli_core imports:
    hp(pixels) - convert pixels to hp
    is_running('rack') - predicate for OS-level process running, default "rack"



In [2]:
from pprint import pprint as pp
from typing import *
from hashlib import sha256
from pathlib import Path
from time import sleep
import os, json, datetime, copy, mido, shutil

In [3]:
rackdir = '/Users/dirkleas/Desktop/h4x/_queue_/vcv/Rack'
# rackdir = '/Users/dirkleas/Documents/Rack'
catalog = json.load(open(rackdir + '/catalog.json'))
catalog_partial = json.load(open(rackdir + '/catalog.partial.json'))
print(f'VCV Rack running from {rackdir}' + (' -- fork FTW!!!' if rackdir.find('Doc') == -1 else ''))

VCV Rack running from /Users/dirkleas/Desktop/h4x/_queue_/vcv/Rack -- fork FTW!!!


First things first, lets create some catalog support infrastructure. In a perfect world, we'd be able to do everyting catalog related from a plugin module, all hands free. Unfortunately, since you can't instantiate plugins outside the main Rack thread, requiring a fork, here are some non-fork support functions relating to the catalog based around this catch-all data model stashed in `catalog.index.json`:

```
{
    sep: SEPARATOR', # separator for PLUGIN composition
    rackdir: DIR, catalog: FILENAME, timestamp: LAST_MOD_TIMESTAMP,
    plugins: {PLUGIN: (INDEX, DIR_HASH)}, pluginCount: 99, # plugin slug, hash used to track plugin updates
    modules: {MODULE: (INDEX, HASH)}, moduleCount: 99, # plugin<sep>module slugs, hash for screenshots filenames
    iterator: [(PLUGIN_SLUG, VERSION, MODULE_SLUG)], # catalog.partial.json order
    faults: [(PLUGIN_SLUG, VERSION, MODULE_SLUG)]
}
```

The plugins and modules indexes can be used to grab a particular plugin/module quickly for targetetd processing. The iterator can be use for processing all the modules, and since it's an ordered list, handy for partial processing restarts (e.g. generating screenshots, patches across the module corpus, etc.). Finally, the plugin hash allows for identifying distribution changes allowing catalog maintenance on an as-needed basis -- the other option is to just look for the particular cataloging asset (e.g. screenshot, instance metadata, etc.).

Further, this `catalog.index.json` can be used to minimize catalog processing by identifying plugins that have changed -- theoretically, evaluate either:

1. file system changes
1. plugin/module slug + plugin version changes

The advatages of the first option is that 1) it guards against plugin developers failing to update version for each release, though it requires multiple filesystem hits for each plugin; and 2) it can be resolved without any Rack code. Maintaining a `catalog.index.json` as a simple list of plugin hashes will allow quick testing for changes -- simply convert to set/list and test for hash inclusion -- if it's there, nothing changed. `catalog.index.json` can be maintained by serializing on every catalog run, or on demand.

While the following implementation is in python, a compatible c++ POC is availble via a `cstat` project pending integration into various `catalog()` implementations.

Note: when running `indexer()` on a large pool of plugins, it's probable that segfaults might occur. To resolve, when a crash occurs do the following:

1. review stacktrace to identify offending module
1. create a patch with `DLwigglz r4xh4x` and the offending module and any previous offenders from this process
1. generate `patch.json` via `DLwigglz r4xh4x` by clicking the `patch` button
1. run `p2f` to generate `faults.json`

These faulty modules will be omitted from the index `iterator` section, and added to a special `faults` section of the same format.

In [56]:
def hash_plugin_module(ps: str, pv: str, ms: str) -> str:
    'hash plugin slug/version, model slug'
    return(sha256(str.encode('{}|{}|{}'.format(ps, pv, ms))).hexdigest())

def hash_plugin_files(plugin_dir: str) -> str:
    'hash based on recursive glob on formtted stat m_times | cocatenated or None if invalid'
    files = sorted([p for p in list(Path(plugin_dir).glob('**/*'))])
    files = [datetime.datetime.fromtimestamp(f.stat().st_mtime).strftime('%Y%m%d%H%M%S') for f in files]
    if (len(files) == 0): return(None)
    return(sha256(str.encode('|'.join(files))).hexdigest())

sep = ' <>+0+<> '

def indexer(c: Dict, catalog_filename: str, index_filename: str, rackdir: str, sep: str=sep) -> Dict:
    'generate index from partial catalog based on catalog.partial.json ordinalities with faults.json support'
    faults = []
    if Path(rackdir + '/faults.json').exists():
        faults = [[m['plugin'], m['version'], m['model']] for m in json.load(open(rackdir + '/faults.json'))]
    i = {'sep': sep,
         'rackdir': rackdir, 'catalog': catalog_filename, 
         'timestamp': datetime.datetime.fromtimestamp(Path(f'{rackdir}/{catalog_filename}').stat().st_mtime).strftime('%Y%m%d%H%M%S'),
         'plugins': {k['slug']:(i,hash_plugin_files(rackdir+'/plugins/'+k['slug'])) for i,k in enumerate(c['plugins'])},
         'modules': {f"{p['slug']}{sep}{m['slug']}":(i,hash_plugin_module(p['slug'], p['version'], m['slug'])) for p in c['plugins'] \
                     for i,m in enumerate(p['models'])},
         'iterator': [[p['slug'], p['version'], m['slug']] for p in c['plugins'] for m in p['models'] \
                      if not [p['slug'], p['version'], m['slug']] in faults],
         'faults': faults,
         'pluginCount': len(c['plugins']),
         'moduleCount': sum(map(lambda p: p['modelCount'], c['plugins']))}
    json.dump(i, open(rackdir + '/' + index_filename, 'w'), indent=2, ensure_ascii=False)
    return(i)

# try:
#     ci = indexer(catalog, 'catalog.json', 'catalog.index.json', rackdir, sep)
#     print(f"generated index on 'catalog.json' as 'catalog.index.json'")
# except: pass
# try:
#     ci = indexer(catalog_partial, 'catalog.partial.json', 'catalog.partial.index.json', rackdir, sep)
#     print(f"generated index on 'catalog.partial.json' as 'catalog.partial.index.json'")
# except: pass

Here are some covience wrapper functions for invoking `DLwigglz r4xh4x` `catalog (partial)` and `patch` buttons using MIDI to trigger keyboard macros. Reference patches are used from the `./patches/_r4xh4x` folder -- update them as underlying modules/Rack evolve. Currently used modules include the following:

1. DLwigglz r4xh4x
1. Frozen Wasteland SeriouslySlowLFO
1. Bogaudio Offset

The following plumbing assumes some type of MIDI-to-keyboard utility software (e.g. [Keyboard Maestro](https://www.keyboardmaestro.com/main/), et'al) with the following mappings (MIDI note => behavior/keystroke):

1. 69 => CTRL+SHIFT+O (Revert toolbar item hotkey)
1. 70 => set focus to Rack from Chrome (assumes it's running)
1. 71 => set focus to Chrome from Rack (assumes it's running)

This can be refactored as soon as Rack supports an extension API for "main thread" features.

In [59]:
def serialize(patch, seq=1):
    'serialize patch and trigger rack reopen/revert'
    json.dump(patch, open(rackdir + '/patches/_r4xh4x_/_r4xh4x_.vcv', 'w'), indent=2, ensure_ascii=False)
#     if patch['modules']:
    json.dump(patch, open(rackdir + f'/patch{str(seq).zfill(5)}.vcv', 'w'), indent=2, ensure_ascii=False)
    sleep(0.5)
    mido.open_output().send(mido.Message('note_on', note=69)); sleep(0.25) # trigger revert/reOpen
    if os.path.exists(rackdir + '/patch.json'):
        os.rename(rackdir + '/patch.json', rackdir + f'/patch{str(seq).zfill(5)}.json')

def patch_module(ps, v, ms, pos=[0, 0]):
    'generate subpatch module fragment at specified rack position'
    module = {'plugin': '', 'version': '', 'model': '', 'pos': [0, 0], 'params': [], 'data': {}} # pos[col,row]
    module['plugin'] = ps; module['version'] = v; module['model'] = ms; module['pos'] = copy.deepcopy(pos)
    return(module)

def r4xh4x_blank_patch(rackdir: str) -> None:
    'revert to blank patch via keyboard macro/MIDI'
    if not is_running(): # consider starting it if not running...
        print(' *** error, rack is not running -- possibly crashed, please investigate')
        return
    mido.open_output().send(mido.Message('note_on', note=70)); sleep(0.25) # head over to rack
    shutil.copy(rackdir + '/patches/_r4xh4x_/_r4xh4x_blank.vcv', 
                rackdir + '/patches/_r4xh4x_/_r4xh4x_.vcv')
    mido.open_output().send(mido.Message('note_on', note=69)); sleep(0.25) # trigger revert/reOpen
    mido.open_output().send(mido.Message('note_on', note=71)) # back to chrome
# r4xh4x_blank_patch(rackdir)

def r4xh4x_catalog_partial_wrapper(rackdir: str) -> None:
    'invoke DLwigglz r4xh4x catalog(partial) functionality via keyboard macro/MIDI'
    print('r4xh4x_catalog_partial_wrapper: switching to VCV Rack and cycling _r4xh4x_.vcv')
    if not is_running(): # consider starting it if not running...
        print(' *** error, rack is not running -- possibly crashed, please investigate')
        return
    mido.open_output().send(mido.Message('note_on', note=70)); sleep(0.25) # head over to rack
    shutil.copy(rackdir + '/patches/_r4xh4x_/_r4xh4x_catalog_wrapper.vcv', 
                rackdir + '/patches/_r4xh4x_/_r4xh4x_.vcv')
    mido.open_output().send(mido.Message('note_on', note=69)); sleep(0.25) # trigger revert/reOpen
    r4xh4x_blank_patch(rackdir)
    print('  partial catalog generated, see catalog.partial.json')
    mido.open_output().send(mido.Message('note_on', note=71)) # back to chrome
r4xh4x_catalog_partial_wrapper(rackdir)
catalog_partial = json.load(open(rackdir + '/catalog.partial.json'))
ci = indexer(catalog_partial, 'catalog.partial.json', 'catalog.partial.index.json', rackdir, sep)
print('generated catalog.partial.index.json from catalog.partial')

# *** VALLEY BLOWS!!! create modules list with just their stuff
def r4xh4x_patch_wrapper(modules: List, limit: int=50) -> None:
    'invoke DLwigglz r4xh4x "patch" on generated modules patch via keyboard macro/midi'
    for p in Path(rackdir).glob('patch*.*'): p.unlink() # zap patch files from prior runs
    print('r4xh4x_patch_wrapper: switching to VCV Rack and cycling _r4xh4x_.vcv')
    if not is_running(): # consider starting it if not running...
        print(' *** error, rack is not running -- possibly crashed, please investigate')
        return
    mido.open_output().send(mido.Message('note_on', note=70)); sleep(0.25) # head over to rack
    patch = json.load(open(rackdir + '/patches/_r4xh4x_/_r4xh4x_patch_wrapper.vcv'))
    reference_module_count = len(patch['modules']) # always keep reference modules when iterating
    idx = 0; seq = 0
    for plugin,version,module in modules:
        if idx == limit: # better serialize patch
            serialize(patch, seq)
            del patch['modules'][reference_module_count:] # delete iteration modules
            idx = 0; seq += 1
        patch['modules'].append(patch_module(plugin, version, module, [0, 1]))
        idx += 1
    if len(patch['modules']) > reference_module_count: serialize(patch, seq)
    r4xh4x_blank_patch(rackdir)
    print('  automatic patch generation complete, see patch*.json files')
    mido.open_output().send(mido.Message('note_on', note=71)) # back to chrome
r4xh4x_patch_wrapper(ci['iterator'])

def r4xh4x_patch_exerciser(rackdir: str) -> None:
    'qa exerciser to reload existing patch99999.vcv test patches from r4xh4x_patch_wrapper to reproduce segfaults'
    print('r4xh4x_patch_exerciser: switching to VCV Rack and cycling _r4xh4x_.vcv')
    if not is_running(): # consider starting it if not running...
        print(' *** error, rack is not running -- possibly crashed, please investigate')
        return
    patches = [f"patch{str(x).zfill(5)}" for x in range(len(list(Path(rackdir).glob('patch*.vcv')))-1)]
    patches = list(map(lambda p: json.load(open(rackdir + f'/{p}.vcv')), patches))
    mido.open_output().send(mido.Message('note_on', note=70)); sleep(0.25) # head over to rack
    for x in range(3):
        for i,p in enumerate(patches):
            if not is_running(): # consider starting it if not running...
                print(' *** error, rack is not running -- possibly crashed, please investigate')
                return
            serialize(p, i)
    r4xh4x_blank_patch(rackdir)
    print('  patch*.json patches exercised')
    mido.open_output().send(mido.Message('note_on', note=71)) # back to chrome
# r4xh4x_patch_exerciser(rackdir)

print(); print(ci['pluginCount'], 'plugins with', ci['moduleCount'], 'modules installed in', rackdir)

r4xh4x_catalog_partial_wrapper: switching to VCV Rack and cycling _r4xh4x_.vcv
  partial catalog generated, see catalog.partial.json
generated catalog.partial.index.json from catalog.partial
r4xh4x_patch_wrapper: switching to VCV Rack and cycling _r4xh4x_.vcv
 *** error, rack is not running -- possibly crashed, please investigate
  automatic patch generation complete, see patch*.json files

110 plugins with 905 modules installed in /Users/dirkleas/Desktop/h4x/_queue_/vcv/Rack


Now we can grab geometry for a given module from it's `r4xhrx patch` serialized output -- still need to figure out whether to keep both `catalog.json` and `catalog.partial.json` around now that we can generate a full `catalog.json` fork-free.

In [6]:
def patch_modules(patch: Dict) -> List:
    'unique sorted list of modules from patch (case insensitive)'
    return(sorted(list(set([(m['plugin'], m['version'], m['model']) for m in patch['modules']])), key=lambda s: s[0].lower()))

def module_nonpartials_from_patch(ps: str, v: str, ms: str, patch: Dict) -> Dict:
    'parse specified module form r4xh4x serialized patch, None if not found'
    module = [m for m in patch['modules'] if m['plugin'] == ps and m['version'] == v and m['model'] == ms]
    if module: [module[0].pop(k) for k in ['plugin', 'version', 'model', 'pos']]
    return(module if len(module) == 1 else None)

patch = json.load(open(rackdir + '/patch00000.json'))
modules = patch_modules(patch)
p,v,m = modules[0]
pp(module_nonpartials_from_patch(p, v, m, patch))
pp(modules)

[{'data': {'shouldTriggerOnLoad': False, 'triggerOnLoad': True},
  'height': 380,
  'inputCount': 7,
  'outputCount': 8,
  'paramCount': 14,
  'params': [{'paramId': 0, 'value': 0.0},
             {'paramId': 1, 'value': 0.119999997},
             {'paramId': 2, 'value': 0.319999993},
             {'paramId': 3, 'value': 0.5},
             {'paramId': 4, 'value': 0.319999993},
             {'paramId': 5, 'value': 0.449999988},
             {'paramId': 6, 'value': 1.0},
             {'paramId': 7, 'value': 1.0},
             {'paramId': 8, 'value': 1.0},
             {'paramId': 9, 'value': 0.0},
             {'paramId': 10, 'value': 1.0},
             {'paramId': 11, 'value': 1.0},
             {'paramId': 12, 'value': 1.0},
             {'paramId': 13, 'value': 1.0}],
  'width': 225}]
[('Bogaudio', '0.6.7', 'Bogaudio-DADSRHPlus'),
 ('Bogaudio', '0.6.7', 'Bogaudio-Noise'),
 ('Bogaudio', '0.6.7', 'Bogaudio-VCA'),
 ('Bogaudio', '0.6.7', 'Bogaudio-Switch'),
 ('Bogaudio', '0.6.7', 'Bogaudi